Zach Kelly

## Research question/interests

The objective behind this repository is to train a machine learning model using a set of fighter statistics to predict score outcomes of future ufc bouts. 

The desired learning outcomes are the following:
learn how to handle large data sets with python
gain experince applying machine learning algorithms
further understand the challenges and limitations of training a successful machine learning model. 

In [ ]:
import numpy as np
import pandas as pd

p_data = pd.read_csv("raw_data.csv")